### Text Selection Data Merge

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Intersect"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0 native word start index
word_end = 45000  # 28 native word end index

# word all usage in twogram
word_use_num_min = 1  # word usage in selected twograms 
word_use_num_max = 2

# n gram select
twogram_select_start = 0
twogram_select_end = 10000

threegram_select_start = 0
threegram_select_end = 10000

fourgram_select_start = 0
fourgram_select_end = 10000

fivegram_select_start = 0
fivegram_select_end = 10000

sixgram_select_start = 0
sixgram_select_end = 10000

sevengram_select_start = 0
sevengram_select_end = 10000

eightgram_select_start = 0
eightgram_select_end = 10000

ninegram_select_start = 0
ninegram_select_end = 10000

tengram_select_start = 0
tengram_select_end = 10000


# prefix suffix file
prefix_suffix = False  # True, False: True for adding prefix suffix word
native_word = True # True for adding native word

# adding output file extention
if prefix_suffix & native_word:
    file_ext = "5"
elif (not prefix_suffix) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              
    
# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"{file_ext}")

6


In [4]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Text Selection/Data/2-Text Selection Data Merge"

Path(path).mkdir(parents=True, exist_ok=True)

In [5]:
def strip(df):
    for i in df.columns:
        new_name = i.strip()
        df.rename(columns={f"{i}":f"{new_name}"}, inplace=True)
        df[f"{new_name}"] = df[f"{new_name}"].apply(lambda x: x.strip())
    
    return df

In [6]:
def word_count_result(df,column_list): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    
    return  df_word_count

In [7]:
def word_in_wordgroup(source_word_list, df_target, target_column, simple=False):

    '''word_in_wordgroup(not_in_sent_word_list, df_youtube_sent_select, "search_string", 5, simple=False)\n
       source_word_list is searching word list\n
       df_target is dataframe, target_column are dataframe column string name\n
       word_sample_num is searching sample number.
       simple use for all column result or only target column result 
    '''
    if simple:
        df_select = df_target[[f"{target_column}"]].dropna()
    else:
        df_select = df_target
        
    df_result = pd.DataFrame()
    for i in source_word_list:
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)]   
        except:
            pass        
        word_in_word_cluster.insert(0,"word",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

In [8]:
def word_in_wordgroup_simple(df, source_column, target_column):

    '''word_in_wordgroup(df, "word", "twogram"):
       df is dataframe, source_column and target_column are 
       dataframe column string name. source_column convert list
       values that are in target column.
    '''
    
    df_select = df[[f"{target_column}"]].dropna()
    df_result = pd.DataFrame()
    for i in df[f"{source_column}"].dropna():
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)]   
        except:
            pass        
        word_in_word_cluster.insert(0,f"{source_column}",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

#### Text Data Read

In [9]:
df_word_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Word_Selected_45000.xlsx")
df_word_select

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
44995,hayırbu,871
44996,konferansına,871
44997,kardeşisin,871
44998,kaldırıyorsun,871


In [10]:
df_twogram_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Twogram_Selected_10000.xlsx")
df_twogram_select

,twogram,frequency
0,bir şey,859944
1,değil mi,585879
2,ben de,377765
3,teşekkür ederim,370619
4,ne oldu,322758
...,...,...
9995,çok acıktım,1317
9996,etmek yok,1317
9997,veya bir,1317
9998,evli misiniz,1316


In [11]:
df_threegram_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Threegram_Selected_10000.xlsx")
df_threegram_select

,threegram,frequency
0,bir şey yok,113165
1,bir şey var,110455
2,bu da ne,89463
3,her şey yolunda,75968
4,başka bir şey,75193
...,...,...
9995,para var mı,701
9996,neden benim için,701
9997,neler oluyor be,701
9998,benim babam değil,701


In [12]:
df_fourgram_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Fourgram_Selected_10000.xlsx")
df_fourgram_select

,fourgram,frequency
0,bir şey var mı,41773
1,her şey yolunda mı,31126
2,burada ne işin var,21993
3,bir sorun mu var,21423
4,ben de seni seviyorum,17338
...,...,...
9995,bir iş görüşmem var,177
9996,için biraz daha zaman,177
9997,iyi olacak tamam mı,177
9998,buna nasıl cesaret edersin,177


In [13]:
df_fivegram_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Fivegram_Selected_10000.xlsx")
df_fivegram_select

,fivegram,frequency
0,başka bir şey var mı,14104
1,bu da ne demek oluyor,10205
2,o kadar da kötü değil,7012
3,sence de öyle değil mi,6305
4,sana bir şey sorabilir miyim,6224
...,...,...
9995,neden kıza onu sevdiğini söylemiyorsun,66
9996,ve şu resme bir bak,66
9997,beni johnnynin bir dostu yolladı,66
9998,bayan teğmen dane ne yaptılar,66


In [14]:
df_sixgram_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Sixgram_Selected_10000.xlsx")
df_sixgram_select

,sixgram,frequency
0,sana söylemem gereken bir şey var,3646
1,bir iki üç dört beş altı,2025
2,söylemek istediğin bir şey var mı,1794
3,daha iyi bir fikrin var mı,1792
4,bunun iyi bir fikir olduğunu sanmıyorum,1743
...,...,...
9995,kurtarmak için basit oyunumu oynaman gerekecek,29
9996,bir şey olduğunu bilen bir adamsın,29
9997,artık yaşlı olduğumdan geceleri dışarı çıkmaya,29
9998,onu haklamak istiyorsan buyur bitir işini,29


In [15]:
df_sevengram_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Sevengram_Selected_10000.xlsx")
df_sevengram_select

,sevengram,frequency
0,bir iki üç dört beş altı yedi,1455
1,iki üç dört beş altı yedi sekiz,1080
2,bana söylemek istediğin bir şey var mı,752
3,yedi altı beş dört üç iki bir,596
4,sekiz yedi altı beş dört üç iki,551
...,...,...
9995,demek istediğini anlıyorum tamam parayı o aldı,23
9996,sizin gibi böyle yakışıklı genç biri için,23
9997,kıçını halledecek olan bir kız mıydı yani,23
9998,kocanın sana altın bir kolye aldığını sanıp,23


In [16]:
df_eightgram_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Eightgram_Selected_10000.xlsx")
df_eightgram_select

,eightgram,frequency
0,bir iki üç dört beş altı yedi sekiz,1060
1,dokuz sekiz yedi altı beş dört üç iki,513
2,sekiz yedi altı beş dört üç iki bir,497
3,iki üç dört beş altı yedi sekiz dokuz,411
4,üç dört beş altı yedi sekiz dokuz on,329
...,...,...
9995,başka bir yerde de patlama olacakmış anlıyor m...,18
9996,uygulama komitesi mi yoksa beni kullanacak ada...,18
9997,tuhaf bir şey değil mi insanın kendini suikastçı,18
9998,hayatındaki en büyük an dostum ve bir yerde,18


In [17]:
df_ninegram_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Ninegram_Selected_10000.xlsx")
df_ninegram_select

,ninegram,frequency
0,dokuz sekiz yedi altı beş dört üç iki bir,463
1,bir iki üç dört beş altı yedi sekiz dokuz,405
2,iki üç dört beş altı yedi sekiz dokuz on,287
3,on dokuz sekiz yedi altı beş dört üç iki,277
4,ve sen herkesin seni tanıdığı o yerde olmak is...,134
...,...,...
9995,sabah yedide buraya gel ve birlikte rakun avın...,13
9996,bunun ne anlama geldiği hakkında bir fikrin va...,13
9997,bence sen böyle bir kurumu idare edebilecek bi...,13
9998,farklı bir şey yapması için bir neden var mı,13


In [18]:
df_tengram_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Tengram_Selected_10000.xlsx")
df_tengram_select

,tengram,frequency
0,umutsuzca birinin arkasından koşuyorum daha gü...,76
1,gece gündüz çifte park edilir insanlar naber k...,70
2,annem bana bazı şeyleri anlatmak için mutlaka ...,65
3,fakat dünyanın ona en çok ihtiyaç duyduğu anda...,64
4,zamanımızı geri almak istiyorum ama şans ve ka...,58
...,...,...
9991,mahkemeye gelip cinayet işlediği sırada beniml...,8
9992,bu olay yüzünden senin yüzünden pek çok insan ...,8
9993,batılı ve arap liderler arasındaki tarihi bir ...,8
9994,başımı belaya sokmam ve bir daha beni düşünmek...,8


#### Text Data Word-Word Group In Word Group

In [19]:
word_list = df_word_select["word"].to_list()
twogram_list = df_twogram_select["twogram"].to_list()
threegram_list = df_threegram_select["threegram"].to_list()
fourgram_list = df_fourgram_select["fourgram"].to_list()
fivegram_list = df_fivegram_select["fivegram"].to_list()
sixgram_list = df_sixgram_select["sixgram"].to_list()
sevengram_list = df_sevengram_select["sevengram"].to_list()
eightgram_list = df_eightgram_select["eightgram"].to_list()
ninegram_list = df_ninegram_select["ninegram"].to_list()
tengram_list = df_tengram_select["tengram"].to_list()

##### Word

In [20]:
df_word_in_two = word_in_wordgroup(word_list, df_twogram_select, "twogram", simple=True)
df_word_in_two

,word,twogram
0,bir,bir şey
1,bir,bu bir
2,bir,bir dakika
3,bir,bir şeyler
4,bir,başka bir
...,...,...
19911,delirdiniz,delirdiniz mi
19912,nyora,nyora duzenleme
19913,duzenleme,nyora duzenleme
19914,shoukanjuu,to shoukanjuu


In [21]:
df_word_in_two["word"].nunique()

3356

##### Twogram

In [22]:
df_two_in_three = word_in_wordgroup(twogram_list, df_threegram_select, "threegram", simple=True)
df_two_in_three.rename(columns={"word":"twogram"}, inplace=True)
df_two_in_three

,twogram,threegram
0,bir şey,bir şey yok
1,bir şey,bir şey var
2,bir şey,başka bir şey
3,bir şey,bir şey değil
4,bir şey,bir şey mi
...,...,...
16488,olduğunu gördün,olduğunu gördün mü
16489,olduğunu gördün,ne olduğunu gördün
16490,bile bilmiyoruz,olduğunu bile bilmiyoruz
16491,iyi olmamıştım,daha iyi olmamıştım


In [35]:
df_two_in_three_list = df_two_in_three.groupby(["twogram"])["threegram"].apply(list).reset_index(name="threegram")
df_two_in_three_list

,twogram,threegram
0,acayip bir,[acayip bir şey]
1,acele edin,"[hadi acele edin, lütfen acele edin, haydi ace..."
2,acele et,"[hadi acele et, acele et ve, haydi acele et, l..."
3,acele etsen,[acele etsen iyi]
4,acil bir,"[acil bir durum, bu acil bir]"
...,...,...
5696,şunu söyleyeyim,[sana şunu söyleyeyim]
5697,şöyle bir,[şöyle bir şey]
5698,şükürler olsun,"[tanrıya şükürler olsun, şükürler olsun ki]"
5699,şüphe yok,"[hiç şüphe yok, buna şüphe yok]"


In [23]:
df_two_in_four = word_in_wordgroup(twogram_list, df_fourgram_select, "fourgram", simple=True)
df_two_in_four.rename(columns={"word":"twogram"}, inplace=True)
df_two_in_four

,twogram,fourgram
0,bir şey,bir şey var mı
1,bir şey,gereken bir şey var
2,bir şey,başka bir şey var
3,bir şey,bir şey daha var
4,bir şey,bir şey mi var
...,...,...
23396,olduğunu gördün,ne olduğunu gördün mü
23397,bile bilmiyoruz,ne olduğunu bile bilmiyoruz
23398,bile bilmiyoruz,olup olmadığını bile bilmiyoruz
23399,iyi olmamıştım,bu kadar iyi olmamıştım


In [36]:
df_two_in_four_list = df_two_in_four.groupby(["twogram"])["fourgram"].apply(list).reset_index(name="fourgram")
df_two_in_four_list

,twogram,fourgram
0,acele edin,[acele edin acele edin]
1,acele et,[acele et acele et]
2,acele etme,[o kadar acele etme]
3,acele etsen,[acele etsen iyi olur]
4,acil bir,"[bu acil bir durum, acil bir durum var, acil b..."
...,...,...
4622,şunu söyleyeyim,[ama sana şunu söyleyeyim]
4623,şöyle bir,[şöyle bir şey var]
4624,şükürler olsun,"[için sana şükürler olsun, için tanrıya şükürl..."
4625,şüphe yok,"[buna hiç şüphe yok, hiç şüphe yok ki]"


In [24]:
df_two_in_five = word_in_wordgroup(twogram_list, df_fivegram_select, "fivegram", simple=True)
df_two_in_five.rename(columns={"word":"twogram"}, inplace=True)
df_two_in_five

,twogram,fivegram
0,bir şey,başka bir şey var mı
1,bir şey,sana bir şey sorabilir miyim
2,bir şey,yapabileceğim bir şey var mı
3,bir şey,sana bir şey göstermek istiyorum
4,bir şey,söylemem gereken bir şey var
...,...,...
26831,sakın bunu,sakın bunu bir daha yapma
26832,sakın bunu,bir daha sakın bunu yapma
26833,iyi olmamıştım,hiç bu kadar iyi olmamıştım
26834,uzun değil,o kadar da uzun değil


In [37]:
df_two_in_five_list = df_two_in_five.groupby(["twogram"])["fivegram"].apply(list).reset_index(name="fivegram")
df_two_in_five_list

,twogram,fivegram
0,acil bir,"[acil bir durum söz konusu, burada acil bir du..."
1,acil durum,"[ne tür bir acil durum, tıbbi bir acil durum var]"
2,acı verici,[ne kadar acı verici olduğunu]
3,adam benim,[bu adam benim hayatımı kurtardı]
4,adam bir,"[bir adam bir günlük operasyon, bir adam bir i..."
...,...,...
4396,şurada bir,[şurada bir şans küpü olmalı]
4397,şöyle bir,[ama şöyle bir şey var]
4398,şöyle dedi,"[o da bana şöyle dedi, sonra avukat bana şöyle..."
4399,şükürler olsun,"[şükürler olsun bir şeyin yok, bunun için tanr..."


In [25]:
df_two_in_six = word_in_wordgroup(twogram_list, df_sixgram_select, "sixgram", simple=True)
df_two_in_six.rename(columns={"word":"twogram"}, inplace=True)
df_two_in_six

,twogram,sixgram
0,bir şey,sana söylemem gereken bir şey var
1,bir şey,söylemek istediğin bir şey var mı
2,bir şey,daha önce hiç böyle bir şey
3,bir şey,için yapabileceğim bir şey var mı
4,bir şey,sana söylemek istediğim bir şey var
...,...,...
20645,olduğunu gördün,ne kadar kolay olduğunu gördün mü
20646,olduğunu gördün,ne kadar güzel olduğunu gördün mü
20647,veya bir,ayakkabı boyası veya bir parça sakız
20648,veya bir,boyası veya bir parça sakız için


In [38]:
df_two_in_six_list = df_two_in_six.groupby(["twogram"])["sixgram"].apply(list).reset_index(name="sixgram")
df_two_in_six_list

,twogram,sixgram
0,acil bir,"[şu an acil bir durum var, efendim ama yorktan..."
1,acı verici,[bu benim için çok acı verici]
2,adam bir,[adam bir tankın kapısını söküp attı]
3,adam da,"[bu adam da senin istediğini istiyor, adam da ..."
4,adam gibi,"[önceki adam gibi sana dönmek istiyorum, araba..."
...,...,...
3806,şunu da,[siz bunu düşünürken şunu da düşünün]
3807,şunu söyleyeyim,[önce şunu söyleyeyim yaşıyorum ve iyiyim]
3808,şöyle bir,"[orada bu duruma uyan şöyle bir, bu duruma uya..."
3809,şükürler olsun,[elijah muhammed için allaha şükürler olsun]


In [26]:
df_two_in_seven = word_in_wordgroup(twogram_list, df_sevengram_select, "sevengram", simple=True)
df_two_in_seven.rename(columns={"word":"twogram"}, inplace=True)
df_two_in_seven

,twogram,sevengram
0,bir şey,bana söylemek istediğin bir şey var mı
1,bir şey,sizin için yapabileceğim bir şey var mı
2,bir şey,senin için yapabileceğim bir şey var mı
3,bir şey,bana söylemek istediğin bir şey mi var
4,bir şey,için yapabileceğim başka bir şey var mı
...,...,...
18126,veya bir,ayakkabı boyası veya bir parça sakız için
18127,veya bir,boyası veya bir parça sakız için bile
18128,veya bir,açık havada veya bir tarlada kalmalı ve
18129,bu gibi,bu gibi durumlarda genelde ne demek gerekir


In [39]:
df_two_in_seven_list = df_two_in_seven.groupby(["twogram"])["sevengram"].apply(list).reset_index(name="sevengram")
df_two_in_seven_list

,twogram,sevengram
0,acele et,[acele et o yüzden çünkü biliyorum ilerledikçe]
1,acil bir,"[efendim ama yorktan çok acil bir mesaj, ama y..."
2,acil durum,[biliyorum ki acil durum gemisi hazırda beklem...
3,acı verici,"[belki bir an acı verici ama sonra, bir an acı..."
4,adam gibi,"[yiyecek adam gibi bir şey bile yok, arabayı a..."
...,...,...
3893,şu tarafta,"[sokağı şu tarafta beş veya altı sokak, şu tar..."
3894,şuna bir,[şuna bir bak bilirsin genç ve güzel]
3895,şöyle bir,"[bu duruma uyan şöyle bir bölüm var, orada bu ..."
3896,şüphe yok,"[şüphe yok ki gemiyi ve bizim hayatlarımızı, h..."


In [27]:
df_two_in_eight = word_in_wordgroup(twogram_list, df_eightgram_select, "eightgram", simple=True)
df_two_in_eight.rename(columns={"word":"twogram"}, inplace=True)
df_two_in_eight

,twogram,eightgram
0,bir şey,sizin için yapabileceğim başka bir şey var mı
1,bir şey,senin için yapabileceğim başka bir şey var mı
2,bir şey,sana söylemem gereken çok önemli bir şey var
3,bir şey,bana söylemek istediğin başka bir şey var mı
4,bir şey,bu isim sana bir şey ifade ediyor mu
...,...,...
19557,veya bir,ayakkabı boyası veya bir parça sakız için bile
19558,veya bir,veya bir çocuk gelip kitap hakkında sorular so...
19559,veya bir,derine temas eder veya bir nefes alırsan anında
19560,veya bir,et geri döner veya bir buluşma yeri vardır


In [40]:
df_two_in_eight_list = df_two_in_eight.groupby(["twogram"])["eightgram"].apply(list).reset_index(name="eightgram")
df_two_in_eight_list

,twogram,eightgram
0,acil bir,"[efendim ama yorktan çok acil bir mesaj var, a..."
1,acil durum,[hey bayım öyle acil durum ipini çekip trenden...
2,acı verici,"[belki bir an acı verici ama sonra uyku, bu du..."
3,adam benim,[onun yasını tuttuğu adam benim en yakın dostu...
4,adam bir,"[ama adam bir kere omzuna kartalı taktı mı, ad..."
...,...,...
4211,şunlara bir,[siz ikiniz şunlara bir göz atın bakalım orada...
4212,şurada bir,[mektubun üzerinde şurada bir parça parmak izi...
4213,şöyle bir,"[orada bu duruma uyan şöyle bir bölüm var, zor..."
4214,şöyle dedi,[ufak bir herif bana geldi ve şöyle dedi]


In [28]:
df_two_in_nine = word_in_wordgroup(twogram_list, df_ninegram_select, "ninegram", simple=True)
df_two_in_nine.rename(columns={"word":"twogram"}, inplace=True)
df_two_in_nine

,twogram,ninegram
0,bir şey,bu iyi bir şey mi kötü bir şey mi
1,bir şey,kılıcını ve onun kılıcını düşün başka bir şey ...
2,bir şey,daha önce hiç bu kadar güzel bir şey görmemiştim
3,bir şey,bir şey almaya dönen olması ihtimaline karşı g...
4,bir şey,bildiğimiz bir şey varsa o da bu arabayı isted...
...,...,...
22039,veya bir,bir şey olursa veya bir şey sorman gerekirse ara
22040,veya bir,peki konuşmaya veya bir şekilde iletişim kurma...
22041,veya bir,resepsiyon bölümünde bekleyebilirsiniz veya bi...
22042,bu gibi,ve bu gibi yerlerde böyle olay çıkarmak size hiç


In [41]:
df_two_in_nine_list = df_two_in_nine.groupby(["twogram"])["ninegram"].apply(list).reset_index(name="ninegram")
df_two_in_nine_list

,twogram,ninegram
0,acele edin,[emrinizi yerine getireceğim ama acele edin lü...
1,acele et,[acele et ve sıkı bir yumrukla bitir onun işini]
2,acelem var,[bir sorun yok dedim sadece acelem var ve beni]
3,acil bir,[afedersiniz efendim ama yorktan çok acil bir ...
4,acil durum,[acil bir durum olduğunda gerçekten acil durum...
...,...,...
4687,şurada bir,[şurada bir yerlerde ben doğdum ve şurada bir ...
4688,şuraya bak,[şuraya bak kardeş bebeğim benim gördüğümü sen...
4689,şöyle dedi,[gün ufak bir herif bana geldi ve şöyle dedi]
4690,şükürler olsun,[sana şükürler olsun ki kral ve kraliçe krallı...


In [29]:
df_two_in_ten = word_in_wordgroup(twogram_list, df_tengram_select, "tengram", simple=True)
df_two_in_ten.rename(columns={"word":"twogram"}, inplace=True)
df_two_in_ten

,twogram,tengram
0,bir şey,bana sadece içinde şeker olmayan bir şey verin...
1,bir şey,niye konuşayım salaklar hayır bugün bana bir ş...
2,bir şey,efendi ani bu isim size bir şey ifade ediyor mu
3,bir şey,çünkü o zaman gerçekten bir şey olursa insanla...
4,bir şey,bir insan bir şeye bakar ve ondan bir şey çıka...
...,...,...
24878,baylar ve,baylar ve bayanlar bu bugüne kadar yaptığım en...
24879,etmek yok,rahatsız etmek yok hiçbir şeye dokunmak yok yo...
24880,veya bir,babam her kimse bir saatliğine veya bir aylığı...
24881,veya bir,aşk tek gecelik veya bir iki haftalık bir olay...


In [42]:
df_two_in_ten_list = df_two_in_ten.groupby(["twogram"])["tengram"].apply(list).reset_index(name="tengram")
df_two_in_ten_list

,twogram,tengram
0,acele edin,[emrinizi yerine getireceğim ama acele edin lü...
1,acele et,[eğer buraya gelmezsen iki tane daha olmak üze...
2,acil bir,[özür dilerim efendim ama yorktan çok acil bir...
3,acil durum,[ama sakın unutma her zaman bir acil durum pla...
4,acı verici,[bu durumun sizin için nasıl acı verici olduğu...
...,...,...
5351,şöyle bir,[düşündüm de yarın şöyle bir uğrayıp onlara bi...
5352,şöyle dedi,[bir gün ufak bir herif bana geldi ve şöyle de...
5353,şöyle yapacağız,[bak şöyle yapacağız sen kızlara erkenden kalk...
5354,şükürler olsun,[şükürler olsun ki o zamandan bu yana uzun yol...


In [30]:
df_two_three_merge = pd.merge(df_twogram_select, df_two_in_three, how="left", on="twogram")
df_two_three_merge.drop_duplicates(inplace=True)
df_two_three_merge

,twogram,frequency,threegram
0,bir şey,859944,bir şey yok
1,bir şey,859944,bir şey var
2,bir şey,859944,başka bir şey
3,bir şey,859944,bir şey değil
4,bir şey,859944,bir şey mi
...,...,...,...
20787,çok acıktım,1317,NaN
20788,etmek yok,1317,NaN
20789,veya bir,1317,NaN
20790,evli misiniz,1316,NaN


In [31]:
df_two_three_merge["twogram"].nunique()

10000

In [49]:
dfs2 = [df_twogram_select, df_two_in_three_list, df_two_in_four_list, df_two_in_five_list, df_two_in_six_list, df_two_in_seven_list, df_two_in_eight_list, df_two_in_nine_list, df_two_in_ten_list]

In [52]:
df_twogram_merge = reduce(lambda  left,right: pd.merge(left,right, on=['twogram'], how='left'), dfs2)
df_twogram_merge

,twogram,frequency,threegram,fourgram,fivegram,sixgram,sevengram,eightgram,ninegram,tengram
0,bir şey,859944,"[bir şey yok, bir şey var, başka bir şey, bir ...","[bir şey var mı, gereken bir şey var, başka bi...","[başka bir şey var mı, sana bir şey sorabilir ...","[sana söylemem gereken bir şey var, söylemek i...","[bana söylemek istediğin bir şey var mı, sizin...",[sizin için yapabileceğim başka bir şey var mı...,"[bu iyi bir şey mi kötü bir şey mi, kılıcını v...",[bana sadece içinde şeker olmayan bir şey veri...
1,değil mi,585879,"[öyle değil mi, biliyorsun değil mi, güzel değ...","[de öyle değil mi, bir şey değil mi, çok güzel...","[sence de öyle değil mi, sizce de öyle değil m...","[bu iyi bir şey değil mi, ne demek istediğimi ...","[o kadar da kötü değildi değil mi, o kadar da ...",[bir kadının şarkı söylemesi oldukça şaşırtıcı...,[evimde bir kadının şarkı söylemesi oldukça şa...,[ama bugün yeni bir maç var galiba değil mi ma...
2,ben de,377765,"[ben de seni, ben de öyle, ve ben de, evet ben...","[ben de seni seviyorum, ben de öyle düşünmüştü...","[ben de seni gördüğüme sevindim, evet ben de ö...","[ben de en az senin kadar, ben de sana aynı şe...","[ben de sizi karı koca ilan ediyorum, ben de o...",[siobhan intihar etti ben de onun yerine geçti...,[siz formları alırsınız ben de onları hazırlar...,[ne dediğimi duymadın mı evlat ben de ona evet...
3,teşekkür ederim,370619,"[çok teşekkür ederim, için teşekkür ederim, te...","[için çok teşekkür ederim, geldiğiniz için teş...","[her şey için teşekkür ederim, zaman ayırdığın...","[her şey için çok teşekkür ederim, beni kabul ...",[olmama izin verdiğin için tekrar teşekkür ede...,[gibi olmama izin verdiğin için tekrar teşekkü...,[kendim gibi olmama izin verdiğin için tekrar ...,[önce parfüm mağazasından alınmış video kaydı ...
4,ne oldu,322758,"[sonra ne oldu, ne oldu sana, sana ne oldu, ne...","[bil bakalım ne oldu, sana ne oldu böyle, ne o...","[az önce ne oldu öyle, sonra ne oldu biliyor m...","[ne oldu bir sorun mu var, az önce ne oldu bil...",[gelecek hakkındaki bilgileri reddetme konuşma...,[insanlık nereye gidiyoruz ve niçin sorularına...,[ya insanlık nereye gidiyoruz ve niçin sorular...,[burnuna ne oldu gittes yoksa kapı yerine baca...
...,...,...,...,...,...,...,...,...,...,...
9995,çok acıktım,1317,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9996,etmek yok,1317,NaN,NaN,[kimseye tek kelime etmek yok],NaN,NaN,NaN,NaN,[rahatsız etmek yok hiçbir şeye dokunmak yok y...
9997,veya bir,1317,NaN,NaN,NaN,"[ayakkabı boyası veya bir parça sakız, boyası ...","[ayakkabı boyası veya bir parça sakız için, bo...",[ayakkabı boyası veya bir parça sakız için bil...,[her kimse bir saatliğine veya bir aylığına da...,[babam her kimse bir saatliğine veya bir aylığ...
9998,evli misiniz,1316,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
df_twogram_merge.to_excel(f"{lang_folder.capitalize()}_Twogram_Text_Selection_Result.xlsx", index=False)

##### Threegram

In [43]:
df_three_in_four = word_in_wordgroup(threegram_list, df_fourgram_select, "fourgram", simple=True)
df_three_in_four.rename(columns={"word":"threegram"}, inplace=True)
df_three_in_four

,threegram,fourgram
0,bir şey yok,diye bir şey yok
1,bir şey yok,başka bir şey yok
2,bir şey yok,bir şey yok mu
3,bir şey yok,endişelenecek bir şey yok
4,bir şey yok,korkacak bir şey yok
...,...,...
12449,bir ismi var,bir ismi var mı
12450,aklına gelirdi ki,kimin aklına gelirdi ki
12451,biraz vaktin var,biraz vaktin var mı
12452,benim babam değil,o benim babam değil


In [44]:
df_three_in_four_list = df_three_in_four.groupby(["threegram"])["fourgram"].apply(list).reset_index(name="fourgram")
df_three_in_four_list

,threegram,fourgram
0,acele etsen iyi,[acele etsen iyi olur]
1,acil bir durum,"[bu acil bir durum, acil bir durum var, acil b..."
2,adam değil mi,[bir adam değil mi]
3,adam tam bir,[bu adam tam bir]
4,adama ihtiyacım var,[bir adama ihtiyacım var]
...,...,...
5656,şunu bil ki,[ama şunu bil ki]
5657,şunu söylemek istiyorum,[sadece şunu söylemek istiyorum]
5658,şunu tamam mı,[kes şunu tamam mı]
5659,şöyle bir şey,[şöyle bir şey var]


In [54]:
df_three_in_five = word_in_wordgroup(threegram_list, df_fivegram_select, "fivegram", simple=True)
df_three_in_five.rename(columns={"word":"threegram"}, inplace=True)
df_three_in_five

,threegram,fivegram
0,bir şey yok,biz diye bir şey yok
1,bir şey yok,başka bir şey yok mu
2,bir şey yok,bir şey yok değil mi
3,bir şey yok,bunda utanılacak bir şey yok
4,bir şey yok,bunda yanlış bir şey yok
...,...,...
14848,dışarı çıkmak ister,benimle dışarı çıkmak ister misin
14849,olduğum bir şey,emin olduğum bir şey var
14850,olduğum bir şey,yapmak zorunda olduğum bir şey
14851,olduğum bir şey,zorunda olduğum bir şey var


In [55]:
df_three_in_five_list = df_three_in_five.groupby(["threegram"])["fivegram"].apply(list).reset_index(name="fivegram")
df_three_in_five_list

,threegram,fivegram
0,acil bir durum,"[acil bir durum söz konusu, burada acil bir du..."
1,adam değil mi,[bu o adam değil mi]
2,adama ihtiyacım var,[daha fazla adama ihtiyacım var]
3,adamı daha önce,"[bu adamı daha önce hiç, adamı daha önce hiç g..."
4,adım daha atarsan,[bir adım daha atarsan seni]
...,...,...
4541,şu anda tam,[şu anda tam olmak istediğim]
4542,şu andan itibaren,[şu andan itibaren her şey]
4543,şuna bir bak,"[gel de şuna bir bak, gel ve şuna bir bak]"
4544,şuna ne dersin,[peki ya şuna ne dersin]


In [56]:
df_three_in_six = word_in_wordgroup(threegram_list, df_sixgram_select, "sixgram", simple=True)
df_three_in_six.rename(columns={"word":"threegram"}, inplace=True)
df_three_in_six

,threegram,sixgram
0,bir şey yok,bir şey yok bir şey yok
1,bir şey yok,yok bir şey yok bir şey
2,bir şey yok,bu konuda yapabileceğim bir şey yok
3,bir şey yok,onun için yapabileceğimiz bir şey yok
4,bir şey yok,bu konuda yapabileceğimiz bir şey yok
...,...,...
9406,sizinle tanıştığıma çok,sizinle tanıştığıma çok memnun oldum bay
9407,en iyi şansımız,eve dönmek için en iyi şansımız
9408,dışarı çıkmak ister,bir ara dışarı çıkmak ister misin
9409,olduğum bir şey,ama emin olduğum bir şey var


In [57]:
df_three_in_six_list = df_three_in_six.groupby(["threegram"])["sixgram"].apply(list).reset_index(name="sixgram")
df_three_in_six_list

,threegram,sixgram
0,acil bir durum,"[şu an acil bir durum var, acil bir durum olma..."
1,adam gibi bir,[yiyecek adam gibi bir şey bile]
2,adam tam bir,[bu adam tam bir baş belası]
3,adama ihtiyacım var,"[yardım edecek bir adama ihtiyacım var, bu köş..."
4,adamı daha önce,"[bu adamı daha önce hiç görmedim, bu adamı dah..."
...,...,...
2840,şu anda ona,[şu anda ona bakıyorum bay hopper]
2841,şu anda sana,[şu anda sana gereken biraz sakinleşmek]
2842,şu anda seni,[şu anda seni bu dünyanın dışına]
2843,şu anda tam,[şu anda tam olmak istediğim yerdeyim]


In [58]:
df_three_in_seven = word_in_wordgroup(threegram_list, df_sevengram_select, "sevengram", simple=True)
df_three_in_seven.rename(columns={"word":"threegram"}, inplace=True)
df_three_in_seven

,threegram,sevengram
0,bir şey yok,söylemek istediğin başka bir şey yok mu
1,bir şey yok,bana söylemek istediğin bir şey yok mu
2,bir şey yok,sen ve ben diye bir şey yok
3,bir şey yok,olmadan ikimizin de yapacağı bir şey yok
4,bir şey yok,başka yapabileceğimiz fazla da bir şey yok
...,...,...
5884,zaman daha iyi,zaman daha iyi bir tane bulup tekrar
5885,zaman daha iyi,o zaman daha iyi bir tane bulup
5886,görmek için geldim,gotham citydeki vahşi hayatı görmek için geldim
5887,en iyi şansımız,gemi eve dönmek için en iyi şansımız


In [59]:
df_three_in_seven_list = df_three_in_seven.groupby(["threegram"])["sevengram"].apply(list).reset_index(name="sevengram")
df_three_in_seven_list

,threegram,sevengram
0,acil bir durum,[acil bir durum olmazsa telefona bile cevap]
1,adam gibi bir,[yiyecek adam gibi bir şey bile yok]
2,adam var ya,"[şu adam var ya sonuna gelince hız, adam var y..."
3,adama ihtiyacım var,[bana yardım edecek bir adama ihtiyacım var]
4,ait bir şey,[sanırım sende bana ait bir şey var]
...,...,...
2198,şu andan itibaren,[şu andan itibaren bir takım gibi çalışalım]
2199,şu işi bitirelim,[iç çamaşırı giyin de şu işi bitirelim]
2200,şu ki bu,"[olan şu ki bu sabah saldırıya uğradım, komik ..."
2201,şuna bir bak,[şuna bir bak bilirsin genç ve güzel]


In [60]:
df_three_in_eight = word_in_wordgroup(threegram_list, df_eightgram_select, "eightgram", simple=True)
df_three_in_eight.rename(columns={"word":"threegram"}, inplace=True)
df_three_in_eight

,threegram,eightgram
0,bir şey yok,birbirimiz olmadan ikimizin de yapacağı bir şe...
1,bir şey yok,önce de aşıktım ve yapabileceğim bir şey yok
2,bir şey yok,bana söylemek istediğin başka bir şey yok mu
3,bir şey yok,yani bize faydası dokunacak bir şey yok mu
4,bir şey yok,beklediğim zevkle yapacağım başka bir şey yok bay
...,...,...
5220,zaman daha iyi,zaman daha iyi bir tane bulup tekrar size
5221,görmek için geldim,onu son bir kez daha görmek için geldim
5222,en iyi şansımız,bu gemi eve dönmek için en iyi şansımız
5223,tıpkı sizin gibi,gelecek olan arabayı bekliyordum tıpkı sizin g...


In [61]:
df_three_in_eight_list = df_three_in_eight.groupby(["threegram"])["eightgram"].apply(list).reset_index(name="eightgram")
df_three_in_eight_list

,threegram,eightgram
0,acil bir durum,[acil bir durum olmazsa telefona bile cevap ve...
1,adam gibi bir,[adam gibi bir işleten olsa iyi iş yapar]
2,adam var ya,[şu adam var ya sonuna gelince hız kesti]
3,adama ihtiyacım var,[özel seçilmiş küçük bir gurup adama ihtiyacım...
4,adı var mı,[senin bu özel dünyanın bir adı var mı]
...,...,...
2255,şu işi bitirelim,[temiz iç çamaşırı giyin de şu işi bitirelim]
2256,şu ki ben,"[şu ki ben doktor değilim avukat da değilim, g..."
2257,şu ki bu,"[komik olan şu ki bu sabah saldırıya uğradım, ..."
2258,şuna bir bak,[şuna bir bak bilirsin genç ve güzel bir]


In [62]:
df_three_in_nine = word_in_wordgroup(threegram_list, df_ninegram_select, "ninegram", simple=True)
df_three_in_nine.rename(columns={"word":"threegram"}, inplace=True)
df_three_in_nine

,threegram,ninegram
0,bir şey yok,ölmeden önce de aşıktım ve yapabileceğim bir ş...
1,bir şey yok,sabırsızlıkla beklediğim zevkle yapacağım başk...
2,bir şey yok,beklediğim zevkle yapacağım başka bir şey yok ...
3,bir şey yok,doldurulmuş hayvanlar ve bir fotoğraftan başka...
4,bir şey yok,yalnız yolculuk yazılmış varış noktası hakkınd...
...,...,...
5562,görmek için geldim,sadece onu son bir kez daha görmek için geldim
5563,tıpkı sizin gibi,tıpkı sizin gibi tamamen yeni bir fikir bulmak...
5564,tıpkı sizin gibi,benim de tıpkı sizin gibi büyük büyük büyükbab...
5565,tıpkı sizin gibi,zenciler de tıpkı sizin gibi sürekli birbirler...


In [63]:
df_three_in_nine_list = df_three_in_nine.groupby(["threegram"])["ninegram"].apply(list).reset_index(name="ninegram")
df_three_in_nine_list

,threegram,ninegram
0,acele et ve,[acele et ve sıkı bir yumrukla bitir onun işini]
1,acil bir durum,[acil bir durum olduğunda gerçekten acil durum...
2,adım daha atarsan,[seni vurmak istemiyorum ama bir adım daha ata...
3,ait bir şey,"[bize ait bir şey var ve o bizi çağırıyor, sen..."
4,ait olan bir,[karşı tarafında zamanında bize ait olan bir k...
...,...,...
2517,şu ki ben,[gerçek şu ki ben doktor değilim avukat da değ...
2518,şuna bir bak,[şuna bir bak bilirsin genç ve güzel bir kadın]
2519,şunu bil ki,[şunu bil ki o adamları hayatımda ilk kez gördüm]
2520,şunu söylemeliyim ki,[şunu söylemeliyim ki seninle resmen tanışmak ...


In [64]:
df_three_in_ten = word_in_wordgroup(threegram_list, df_tengram_select, "tengram", simple=True)
df_three_in_ten.rename(columns={"word":"threegram"}, inplace=True)
df_three_in_ten

,threegram,tengram
0,bir şey yok,o ölmeden önce de aşıktım ve yapabileceğim bir...
1,bir şey yok,sabırsızlıkla beklediğim zevkle yapacağım başk...
2,bir şey yok,elimde doldurulmuş hayvanlar ve bir fotoğrafta...
3,bir şey yok,ama kimin öldüğünü umursamıyor çünkü karakter ...
4,bir şey yok,sizi buradayken koruyabilirim ama dışarıdayken...
...,...,...
6236,görmek için geldim,buraya sadece onu son bir kez daha görmek için...
6237,olduğum bir şey,ve emin olduğum bir şey var madam kızınızı çok...
6238,olduğum bir şey,emin olduğum bir şey var ki benim dünyama göre...
6239,iyi olup olmadığını,karım kaza geçirdi ve onun iyi olup olmadığını...


In [65]:
df_three_in_ten_list = df_three_in_ten.groupby(["threegram"])["tengram"].apply(list).reset_index(name="tengram")
df_three_in_ten_list

,threegram,tengram
0,acil bir durum,[dostlar acil bir durum yaşadık ama şimdi her ...
1,acil durumlar için,[babam acil durumlar için bana hong kongda bir...
2,adam değil mi,[yani o hayatta tutmak isteyeceğin türde bir a...
3,ait bir şey,[sende bize ait bir şey var ve o bizi çağırıyo...
4,ait olan bir,[diyorlar ki siz de bana ait olan bir şey varm...
...,...,...
3166,şu ki bu,[gerçek şu ki bu kapıdan girdiğinizde bir şeyi...
3167,şunu söylemek istiyorum,[bir yararı olur mu bilmiyorum ama sana şunu s...
3168,şunu söylemeliyim ki,[şunu söylemeliyim ki bazı hatalar yaptım ve b...
3169,şunu tutar mısın,[bana bir dakika müsaade edin tuvalete gitmeli...


In [66]:
dfs3 = [df_threegram_select, df_three_in_four_list, df_three_in_five_list, df_three_in_six_list, df_three_in_seven_list, df_three_in_eight_list, df_three_in_nine_list, df_three_in_ten_list]

In [67]:
df_threegram_merge = reduce(lambda  left,right: pd.merge(left,right, on=['threegram'], how='left'), dfs3)
df_threegram_merge

,threegram,frequency,fourgram,fivegram,sixgram,sevengram,eightgram,ninegram,tengram
0,bir şey yok,113165,"[diye bir şey yok, başka bir şey yok, bir şey ...","[biz diye bir şey yok, başka bir şey yok mu, b...","[bir şey yok bir şey yok, yok bir şey yok bir ...","[söylemek istediğin başka bir şey yok mu, bana...",[birbirimiz olmadan ikimizin de yapacağı bir ş...,[ölmeden önce de aşıktım ve yapabileceğim bir ...,[o ölmeden önce de aşıktım ve yapabileceğim bi...
1,bir şey var,110455,"[bir şey var mı, gereken bir şey var, başka bi...","[başka bir şey var mı, yapabileceğim bir şey v...","[sana söylemem gereken bir şey var, söylemek i...","[bana söylemek istediğin bir şey var mı, sizin...",[sizin için yapabileceğim başka bir şey var mı...,[ev sahibi babamız için bende özel bir şey var...,[kaosun içinde düzen için mücadele eden tek bi...
2,bu da ne,89463,"[bu da ne demek, bu da ne böyle, bu da ne be, ...","[bu da ne demek oluyor, bu da ne demek şimdi, ...","[bu da ne demek oluyor şimdi, bu da ne demek o...",NaN,NaN,NaN,NaN
3,her şey yolunda,75968,"[her şey yolunda mı, her şey yolunda gidecek, ...","[orada her şey yolunda mı, burada her şey yolu...","[her şey yolunda her şey yolunda, merak etme h...",[helikopterler bizi koruyacak her şey yolunda ...,[her şey yolunda giderse çok yakında bir ev],[her şey yolunda giderse çok yakında bir ev al...,[dostlar acil bir durum yaşadık ama şimdi her ...
4,başka bir şey,75193,"[başka bir şey var, başka bir şey yok, başka b...","[başka bir şey var mı, başka bir şey daha var,...","[istediğin başka bir şey var mı, yapabileceğim...","[için yapabileceğim başka bir şey var mı, bilm...",[sizin için yapabileceğim başka bir şey var mı...,[kılıcını ve onun kılıcını düşün başka bir şey...,[sabırsızlıkla beklediğim zevkle yapacağım baş...
...,...,...,...,...,...,...,...,...,...
9995,para var mı,701,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9996,neden benim için,701,NaN,NaN,NaN,NaN,NaN,NaN,[peki sen neden benim için bu kadar çok şey ya...
9997,neler oluyor be,701,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9998,benim babam değil,701,[o benim babam değil],NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
df_threegram_merge.to_excel(f"{lang_folder.capitalize()}_Threegram_Text_Selection_Result.xlsx", index=False)

##### Fourgram

In [ ]:
df_four_in_five = word_in_wordgroup(fourgram_list, df_fivegram_select, "fivegram", simple=True)
df_four_in_five.rename(columns={"word":"fourgram"}, inplace=True)
df_four_in_five

In [ ]:
df_four_in_five_list = df_four_in_five.groupby(["fourgram"])["fivegram"].apply(list).reset_index(name="fivegram")
df_four_in_five_list

In [ ]:
df_four_in_six = word_in_wordgroup(fourgram_list, df_sixgram_select, "sixgram", simple=True)
df_four_in_six.rename(columns={"word":"fourgram"}, inplace=True)
df_four_in_six

In [ ]:
df_four_in_six_list = df_four_in_six.groupby(["fourgram"])["sixgram"].apply(list).reset_index(name="sixgram")
df_four_in_six_list

In [ ]:
df_four_in_seven = word_in_wordgroup(fourgram_list, df_sevengram_select, "sevengram", simple=True)
df_four_in_seven.rename(columns={"word":"fourgram"}, inplace=True)
df_four_in_seven

In [ ]:
df_four_in_seven_list = df_four_in_seven.groupby(["fourgram"])["sevengram"].apply(list).reset_index(name="sevengram")
df_four_in_seven_list

In [ ]:
df_four_in_eight = word_in_wordgroup(fourgram_list, df_eightgram_select, "eightgram", simple=True)
df_four_in_eight.rename(columns={"word":"fourgram"}, inplace=True)
df_four_in_eight

In [ ]:
df_four_in_eight_list = df_four_in_eight.groupby(["fourgram"])["eightgram"].apply(list).reset_index(name="eightgram")
df_four_in_eight_list

In [ ]:
df_four_in_nine = word_in_wordgroup(fourgram_list, df_ninegram_select, "ninegram", simple=True)
df_four_in_nine.rename(columns={"word":"fourgram"}, inplace=True)
df_four_in_nine

In [ ]:
df_four_in_nine_list = df_four_in_nine.groupby(["fourgram"])["ninegram"].apply(list).reset_index(name="ninegram")
df_four_in_nine_list

In [ ]:
df_four_in_ten = word_in_wordgroup(fourgram_list, df_tengram_select, "tengram", simple=True)
df_four_in_ten.rename(columns={"word":"fourgram"}, inplace=True)
df_four_in_ten

In [ ]:
df_four_in_ten_list = df_four_in_ten.groupby(["fourgram"])["tengram"].apply(list).reset_index(name="tengram")
df_four_in_ten_list

In [ ]:
dfs4 = [df_fourgram_select, df_four_in_five_list, df_four_in_six_list, df_four_in_seven_list, df_four_in_eight_list, df_four_in_nine_list, df_four_in_ten_list]

In [ ]:
df_fourgram_merge = reduce(lambda  left,right: pd.merge(left,right, on=['fourgram'], how='left'), dfs4)
df_fourgram_merge

In [ ]:
df_fourgram_merge.to_excel(f"{lang_folder.capitalize()}_Fourgram_Text_Selection_Result.xlsx", index=False)

##### Fivegram

In [ ]:
df_five_in_six = word_in_wordgroup(fivegram_list, df_sixgram_select, "sixgram", simple=True)
df_five_in_six.rename(columns={"word":"fivegram"}, inplace=True)
df_five_in_six

In [ ]:
df_five_in_six_list = df_five_in_six.groupby(["fivegram"])["sixgram"].apply(list).reset_index(name="sixgram")
df_five_in_six_list

In [ ]:
df_five_in_seven = word_in_wordgroup(fivegram_list, df_sevengram_select, "sevengram", simple=True)
df_five_in_seven.rename(columns={"word":"fivegram"}, inplace=True)
df_five_in_seven

In [ ]:
df_five_in_seven_list = df_five_in_seven.groupby(["fivegram"])["sevengram"].apply(list).reset_index(name="sevengram")
df_five_in_seven_list

In [ ]:
df_five_in_eight = word_in_wordgroup(fivegram_list, df_eightgram_select, "eightgram", simple=True)
df_five_in_eight.rename(columns={"word":"fivegram"}, inplace=True)
df_five_in_eight

In [ ]:
df_five_in_eight_list = df_five_in_eight.groupby(["fivegram"])["eightgram"].apply(list).reset_index(name="eightgram")
df_five_in_eight_list

In [ ]:
df_five_in_nine = word_in_wordgroup(fivegram_list, df_ninegram_select, "ninegram", simple=True)
df_five_in_nine.rename(columns={"word":"fivegram"}, inplace=True)
df_five_in_nine

In [ ]:
df_five_in_nine_list = df_five_in_nine.groupby(["fivegram"])["ninegram"].apply(list).reset_index(name="ninegram")
df_five_in_nine_list

In [ ]:
df_five_in_ten = word_in_wordgroup(fivegram_list, df_tengram_select, "tengram", simple=True)
df_five_in_ten.rename(columns={"word":"fivegram"}, inplace=True)
df_five_in_ten

In [ ]:
df_five_in_ten_list = df_five_in_ten.groupby(["fivegram"])["tengram"].apply(list).reset_index(name="tengram")
df_five_in_ten_list

In [ ]:
dfs5 = [df_fivegram_select, df_five_in_six_list, df_five_in_seven_list, df_five_in_eight_list, df_five_in_nine_list, df_five_in_ten_list]

In [ ]:
df_fivegram_merge = reduce(lambda  left,right: pd.merge(left,right, on=['fivegram'], how='left'), dfs5)
df_fivegram_merge

In [ ]:
df_fivegram_merge.to_excel(f"{lang_folder.capitalize()}_Fivegram_Text_Selection_Result.xlsx", index=False)

##### Sixgram

In [ ]:
df_six_in_seven = word_in_wordgroup(sixgram_list, df_sevengram_select, "sevengram", simple=True)
df_six_in_seven.rename(columns={"word":"sixgram"}, inplace=True)
df_six_in_seven

In [ ]:
df_six_in_seven_list = df_six_in_seven.groupby(["sixgram"])["sevengram"].apply(list).reset_index(name="sevengram")
df_six_in_seven_list

In [ ]:
df_six_in_eight = word_in_wordgroup(sixgram_list, df_eightgram_select, "eightgram", simple=True)
df_six_in_eight.rename(columns={"word":"sixgram"}, inplace=True)
df_six_in_eight

In [ ]:
df_six_in_eight_list = df_six_in_eight.groupby(["sixgram"])["eightgram"].apply(list).reset_index(name="eightgram")
df_six_in_eight_list

In [ ]:
df_six_in_nine = word_in_wordgroup(sixgram_list, df_ninegram_select, "ninegram", simple=True)
df_six_in_nine.rename(columns={"word":"sixgram"}, inplace=True)
df_six_in_nine

In [ ]:
df_six_in_nine_list = df_six_in_nine.groupby(["sixgram"])["ninegram"].apply(list).reset_index(name="ninegram")
df_six_in_nine_list

In [ ]:
df_six_in_ten = word_in_wordgroup(sixgram_list, df_tengram_select, "tengram", simple=True)
df_six_in_ten.rename(columns={"word":"sixgram"}, inplace=True)
df_six_in_ten

In [ ]:
df_six_in_ten_list = df_six_in_ten.groupby(["sixgram"])["tengram"].apply(list).reset_index(name="tengram")
df_six_in_ten_list

In [ ]:
dfs6 = [df_sixgram_select, df_six_in_seven_list, df_six_in_eight_list, df_six_in_nine_list, df_six_in_ten_list]

In [ ]:
df_sixgram_merge = reduce(lambda  left,right: pd.merge(left,right, on=['sixgram'], how='left'), dfs6)
df_sixgram_merge

In [ ]:
df_sixgram_merge.to_excel(f"{lang_folder.capitalize()}_Sixgram_Text_Selection_Result.xlsx", index=False)

##### Sevengram

In [ ]:
df_seven_in_eight = word_in_wordgroup(sevengram_list, df_eightgram_select, "eightgram", simple=True)
df_seven_in_eight.rename(columns={"word":"sevengram"}, inplace=True)
df_seven_in_eight

In [ ]:
df_seven_in_eight_list = df_seven_in_eight.groupby(["sevengram"])["eightgram"].apply(list).reset_index(name="eightgram")
df_seven_in_eight_list

In [ ]:
df_seven_in_nine = word_in_wordgroup(sevengram_list, df_ninegram_select, "ninegram", simple=True)
df_seven_in_nine.rename(columns={"word":"sevengram"}, inplace=True)
df_seven_in_nine

In [ ]:
df_seven_in_nine_list = df_seven_in_nine.groupby(["sevengram"])["ninegram"].apply(list).reset_index(name="ninegram")
df_seven_in_nine_list

In [ ]:
df_seven_in_ten = word_in_wordgroup(sevengram_list, df_tengram_select, "tengram", simple=True)
df_seven_in_ten.rename(columns={"word":"sevengram"}, inplace=True)
df_seven_in_ten

In [ ]:
df_seven_in_ten_list = df_seven_in_ten.groupby(["sevengram"])["tengram"].apply(list).reset_index(name="tengram")
df_seven_in_ten_list

In [ ]:
dfs7 = [df_sevengram_select, df_seven_in_eight_list, df_seven_in_nine_list, df_seven_in_ten_list]

In [ ]:
df_sevengram_merge = reduce(lambda  left,right: pd.merge(left,right, on=['sevengram'], how='left'), dfs7)
df_sevengram_merge

In [ ]:
df_sevengram_merge.to_excel(f"{lang_folder.capitalize()}_Sevengram_Text_Selection_Result.xlsx", index=False)

##### Eightgram

In [ ]:
df_eight_in_nine = word_in_wordgroup(eightgram_list, df_ninegram_select, "ninegram", simple=True)
df_eight_in_nine.rename(columns={"word":"eightgram"}, inplace=True)
df_eight_in_nine

In [ ]:
df_eight_in_nine_list = df_eight_in_nine.groupby(["eightgram"])["ninegram"].apply(list).reset_index(name="ninegram")
df_eight_in_nine_list

In [ ]:
df_eight_in_ten = word_in_wordgroup(eightgram_list, df_tengram_select, "tengram", simple=True)
df_eight_in_ten.rename(columns={"word":"eightgram"}, inplace=True)
df_eight_in_ten

In [ ]:
df_eight_in_ten_list = df_eight_in_ten.groupby(["eightgram"])["tengram"].apply(list).reset_index(name="tengram")
df_eight_in_ten_list

In [ ]:
dfs8 = [df_eightgram_select, df_eight_in_nine_list, df_eight_in_ten_list]

In [ ]:
df_eightgram_merge = reduce(lambda  left,right: pd.merge(left,right, on=['eightgram'], how='left'), dfs8)
df_eightgram_merge

In [ ]:
df_eightgram_merge.to_excel(f"{lang_folder.capitalize()}_Eightgram_Text_Selection_Result.xlsx", index=False)

##### Ninegram

In [ ]:
df_nine_in_ten = word_in_wordgroup(ninegram_list, df_tengram_select, "tengram", simple=True)
df_nine_in_ten.rename(columns={"word":"ninegram"}, inplace=True)
df_nine_in_ten

In [ ]:
df_nine_in_ten_list = df_nine_in_ten.groupby(["ninegram"])["tengram"].apply(list).reset_index(name="tengram")
df_nine_in_ten_list

In [ ]:
dfs9 = [df_ninegram_select, df_nine_in_ten_list]

In [ ]:
df_ninegram_merge = reduce(lambda  left,right: pd.merge(left,right, on=['ninegram'], how='left'), dfs9)
df_ninegram_merge

In [ ]:
df_ninegram_merge.to_excel(f"{lang_folder.capitalize()}_Ninegram_Text_Selection_Result.xlsx", index=False)

##### Text Data Word-Word Group In Word Group Merge

In [ ]:
df_1_2_3_merge = pd.merge(df_word_in_two, df_two_in_three, how="left", on="twogram")
df_1_2_3_merge.drop_duplicates(inplace=True)
df_1_2_3_merge

In [ ]:
df_1_2_3_4_merge = pd.merge(df_1_2_3_merge, df_three_in_four, how="left", on="threegram")
df_1_2_3_4_merge.drop_duplicates(inplace=True)
df_1_2_3_4_merge

In [ ]:
df_1_2_3_4_5_merge = pd.merge(df_1_2_3_4_merge, df_four_in_five, how="left", on="fourgram")
df_1_2_3_4_5_merge.drop_duplicates(inplace=True)
df_1_2_3_4_5_merge

In [ ]:
df_1_2_3_4_5_6_merge = pd.merge(df_1_2_3_4_5_merge, df_five_in_six, how="left", on="fivegram")
df_1_2_3_4_5_6_merge.drop_duplicates(inplace=True)
df_1_2_3_4_5_6_merge

In [ ]:
df_1_2_3_4_5_6_7_merge = pd.merge(df_1_2_3_4_5_6_merge, df_six_in_seven, how="left", on="sixgram")
df_1_2_3_4_5_6_7_merge.drop_duplicates(inplace=True)
df_1_2_3_4_5_6_7_merge

In [ ]:
df_1_2_3_4_5_6_7_8_merge = pd.merge(df_1_2_3_4_5_6_7_merge, df_seven_in_eight, how="left", on="sevengram")
df_1_2_3_4_5_6_7_8_merge.drop_duplicates(inplace=True)
df_1_2_3_4_5_6_7_8_merge

In [ ]:
df_1_2_3_4_5_6_7_8_9_merge = pd.merge(df_1_2_3_4_5_6_7_8_merge, df_eight_in_nine, how="left", on="eightgram")
df_1_2_3_4_5_6_7_8_9_merge.drop_duplicates(inplace=True)
df_1_2_3_4_5_6_7_8_9_merge

In [ ]:
df_1_2_3_4_5_6_7_8_9_10_merge = pd.merge(df_1_2_3_4_5_6_7_8_9_merge, df_nine_in_ten, how="left", on="ninegram")
df_1_2_3_4_5_6_7_8_9_10_merge.drop_duplicates(inplace=True)
df_1_2_3_4_5_6_7_8_9_10_merge

In [ ]:
df_1_2_3_4_5_6_7_8_9_10_merge

In [ ]:
df_1_2_3_4_5_6_7_8_9_10_merge["word"].nunique()

In [ ]:
word_count_result(df_1_2_3_4_5_6_7_8_9_10_merge,["twogram","threegram","fourgram","fivegram","sixgram","sevengram","eightgram","ninegram","tengram"])

In [ ]:
#df_1_2_3_4_5_6_7_8_9_10_merge.to_excel(f"{lang_folder.capitalize()}_Text_Selection_Data.xlsx", index=False)

In [ ]:
#df_1_2_3_4_5_6_7_8_9_10_freq_merge = pd.merge(df_1_2_3_4_5_6_7_8_9_10_merge, df_word_select, how="inner", on="word")
df_1_2_3_4_5_6_7_8_9_10_freq_merge = pd.merge(df_word_select, df_1_2_3_4_5_6_7_8_9_10_merge, how="left", on="word")
df_1_2_3_4_5_6_7_8_9_10_freq_merge.drop_duplicates(inplace=True)
#df_1_2_3_4_5_6_7_8_9_10_freq_merge.sort_values(by="frequency", inplace=True, ascending=False)
df_1_2_3_4_5_6_7_8_9_10_freq_merge

In [ ]:
df_1_2_3_4_5_6_7_8_9_10_freq_merge["word"].nunique()

In [ ]:
df_1_2_3_4_5_6_7_8_9_10_freq_merge["tengram"].nunique()

In [ ]:
df_1_2_3_4_5_6_7_8_9_10_freq_merge["ninegram"].nunique()

In [ ]:
df_1_2_3_4_5_6_7_8_9_10_freq_merge["eightgram"].nunique()

#### Copy Move And Delete

In [69]:
output_file = glob.glob(f"{lang_folder.capitalize()}_*_Text_Selection_Result.xlsx")
#output_file = glob.glob(f"*_Selected_*.xlsx")
output_file

['Turkish_Twogram_Text_Selection_Result.xlsx',
 'Turkish_Threegram_Text_Selection_Result.xlsx']

In [70]:
for l in output_file:
    source = l # source directory
    destination = path
    shutil.copy2(source, destination)

In [71]:
for j in output_file:
    try:
        os.remove(j)
    except:
        pass

#### Temp

In [ ]:
df_text_file = pd.read_excel("Turkish_Text_Selection_Data.xlsx")
df_text_file

In [ ]:
df_text_file["word"].nunique()

In [ ]:
df_text_file.groupby("word")[["twogram"]].agg(lambda x : set(x))

In [ ]:
df_text_file.groupby("twogram")[["threegram"]].agg(lambda x : set(x))

In [ ]:
df_text_file.groupby("threegram")[["fourgram"]].agg(lambda x : set(x))

In [ ]:
df_text_file.groupby("fourgram")[["fivegram"]].agg(lambda x : set(x))

In [ ]:
df_text_file.groupby("fivegram")[["sixgram"]].agg(lambda x : set(x))

In [ ]:
df_test = df_text_file.groupby("fivegram")[["sixgram"]].agg(lambda x : set(x))
df_test.reset_index(inplace=True)
df_test

In [ ]:
fivegram_list = df_test["fivegram"].to_list()

In [ ]:
df_fivegram_select[~df_fivegram_select["fivegram"].isin(fivegram_list)]

In [ ]:
df_twogram_select[df_twogram_select["twogram"]=="söz vermedim"]

In [ ]:
df_threegram_select[df_threegram_select["threegram"]=="öyle bir söz"]

In [ ]:
df_text_file.groupby("sixgram")[["sevengram"]].agg(lambda x : set(x))

In [ ]:
df_text_file.groupby("sevengram")[["eightgram"]].agg(lambda x : set(x))

In [ ]:
df_text_file.groupby("eightgram")[["ninegram"]].agg(lambda x : set(x))

In [ ]:
df_text_file.groupby("ninegram")[["tengram"]].agg(lambda x : set(x))

In [ ]:
df_ngram_concat = pd.concat([df_twogram_select, df_threegram_select, df_fourgram_select, df_fivegram_select], axis=1)
df_ngram_concat

In [ ]:
df_two_in_three = word_in_wordgroup_simple(df_ngram_concat, "twogram","threegram",threegram_sample)
df_two_in_four = word_in_wordgroup_simple(df_ngram_concat, "twogram","fourgram",fourgram_sample)
df_two_in_five = word_in_wordgroup_simple(df_ngram_concat, "twogram","fivegram",fivegram_sample)

In [ ]:
df_twogram_order_join_threegram = df_two_in_three.groupby(["twogram"])["threegram"].apply(", ".join).reset_index()
df_twogram_order_join_fourgram = df_two_in_four.groupby(["twogram"])["fourgram"].apply(", ".join).reset_index()
df_twogram_order_join_fivegram = df_two_in_five.groupby(["twogram"])["fivegram"].apply(", ".join).reset_index()  

In [ ]:
dfs = [df_twogram_order_join_threegram, df_twogram_order_join_fourgram, df_twogram_order_join_fivegram]

In [ ]:
df_ngram_sample_join_merge = reduce(lambda  left,right: pd.merge(left,right, on=['twogram'], how='outer'), dfs)
df_ngram_sample_join_merge.drop_duplicates(inplace=True)
df_ngram_sample_join_merge 

In [ ]:
df_ngram_sample_join_merge = pd.merge(df_ngram_sample_join_merge, df_twogram_select, how="right", on="twogram")
df_ngram_sample_join_merge.drop_duplicates(inplace=True)
df_ngram_sample_join_merge.rename(columns={"frequency":"two_freq"}, inplace=True)
df_ngram_sample_join_merge.sort_values(by="two_freq", ascending=False, inplace=True)
df_ngram_sample_join_merge.reset_index(drop=True, inplace=True)
df_ngram_sample_join_merge

In [ ]:
df_ngram_sample_join_merge.to_excel(f"{lang_folder.capitalize()}_{len(df_twogram_select)}_Twogram_In_{threegram_sample}_Threegram_{fourgram_sample}_\
Fourgram_{fivegram_sample}_Fivegram_Sample_With_{word_end}_Word_Join_Result.xlsx", index=False)

#### Adding Frequency For Ratio

In [ ]:
df_two_in_three_freq = pd.merge(df_two_in_three, df_threegram, how="left", on="threegram")
df_two_in_three_freq.drop_duplicates(inplace=True)
df_two_in_three_freq.rename(columns={"frequency":"three_freq"}, inplace=True)
df_two_in_three_freq.drop(["twogram"], axis=1, inplace=True)
df_two_in_three_freq

In [ ]:
df_two_in_four_freq = pd.merge(df_two_in_four, df_fourgram, how="left", on="fourgram")
df_two_in_four_freq.drop_duplicates(inplace=True)
df_two_in_four_freq.rename(columns={"frequency":"four_freq"}, inplace=True)
df_two_in_four_freq.drop(["twogram"], axis=1, inplace=True)
df_two_in_four_freq

In [ ]:
df_two_in_five_freq = pd.merge(df_two_in_five, df_fivegram, how="left", on="fivegram")
df_two_in_five_freq.drop_duplicates(inplace=True)
df_two_in_five_freq.rename(columns={"frequency":"five_freq"}, inplace=True)
df_two_in_five_freq.drop(["twogram"], axis=1, inplace=True)
df_two_in_five_freq

In [ ]:
df_ngram_sample_concat = pd.concat([df_twogram_select,df_two_in_three_freq, df_two_in_four_freq, df_two_in_five_freq], axis=1)
df_ngram_sample_concat

In [ ]:
df_ngram_sample_concat.to_excel(f"{lang_folder.capitalize()}_{len(df_twogram_select)}_Twogram_In_{threegram_sample}_Threegram_{fourgram_sample}_\
Fourgram_{fivegram_sample}_Fivegram_Sample_With_{word_end}_Word_Frequency_Result.xlsx", index=False)

#### Calculate Result And Select Ratio

In [ ]:
df_ngram_threegram_unique = df_ngram_sample_concat[["threegram","three_freq"]].drop_duplicates()
df_ngram_fourgram_unique = df_ngram_sample_concat[["fourgram","four_freq"]].drop_duplicates()
df_ngram_fivegram_unique = df_ngram_sample_concat[["fivegram","five_freq"]].drop_duplicates()

In [ ]:
twogram_result_freq = df_twogram_select["frequency"].sum()
threegram_result_freq = df_ngram_threegram_unique["three_freq"].sum()
fourgram_result_freq = df_ngram_fourgram_unique["four_freq"].sum()
fivegram_result_freq = df_ngram_fivegram_unique["five_freq"].sum()

In [ ]:
twogram_select_freq = df_twogram.iloc[twogram_select_start:twogram_select_end,]["frequency"].sum()
threegram_select_freq = df_threegram.iloc[threegram_select_start:threegram_select_end,]["frequency"].sum()
fourgram_select_freq = df_fourgram.iloc[fourgram_select_start:fourgram_select_end,]["frequency"].sum()
fivegram_select_freq = df_fivegram.iloc[fivegram_select_start:fivegram_select_end,]["frequency"].sum()

In [ ]:
(twogram_result_freq/twogram_select_freq)*100

In [ ]:
(threegram_result_freq/threegram_select_freq)*100

In [ ]:
(fourgram_result_freq/fourgram_select_freq)*100

In [ ]:
(fivegram_result_freq/fivegram_select_freq)*100

#### Copy Move And Delete

In [ ]:
output_file = glob.glob(f"{lang_folder.capitalize()}_{len(df_twogram_select)}_Twogram_In_{threegram_sample}_Threegram_{fourgram_sample}_\
Fourgram_{fivegram_sample}_Fivegram_Sample_*_Result.xlsx")
output_file

In [ ]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [ ]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass

#### Temp

In [ ]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [ ]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

In [ ]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Intersect"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0 native word start index
word_end = 200  # 28 native word end index

# word all usage in twogram
word_use_num_min = 1  # word usage in selected twograms 
word_use_num_max = 4

In [ ]:
def word_usage_result(word_list, df_target, target_column, word_usage_min, word_usage_max): # word_usage_result(word_list, df_target, target_column, target_opt_column, word_usage_min, word_usage_max)
    '''
    word_usage_result(word_list, df_ngram_pair, "threegram", "frequency", 1, 5) \n
    word_list is a list, df_target is a dateframe, target_column is df_target dataframe target column, \n
    target_opt_column is df_target dataframe opt_target column, \n
    word_usage_min and word_usage_max word usage condition.
    '''    
    word_num_dict = {}
    for i in word_list:
        word_num_dict[f"{i}"] = 0
    
    result_list_select = []
    var_list = []
    for i in range(len(df_target)):
        target_value = df_target.loc[i,f"{target_column}"]
        #opt_value = df_target.loc[i,f"{target_opt_column}"]
        words = word_tokenize(target_value)   
        temp_list = [word for word in words]
        temp_list = temp_list + var_list
        # word count for max
        dict_list_count = Counter(temp_list)
        count_list = list(dict_list_count.values())
        # word count for min
        count_list2 = list(word_num_dict.values())
    
        if any([True if i>word_usage_max else False for i in count_list]) or not(any([True if j<word_usage_min else False for j in count_list2])):
            pass
        else:
            var_list = temp_list
            result_list_select.append([target_value])
            #result_list_select.append([target_value,opt_value])  
    
            for item2 in dict_list_count.items(): 
                word_num_dict[item2[0]] = item2[1]        
    df_result = pd.DataFrame(result_list_select, columns=[f"{target_column}"])
    #df_result = pd.DataFrame(result_list_select, columns=[f"{target_column}",f"{target_opt_column}"])
    #df_result.sort_values(by="frequency", ascending=False, inplace=True)
    df_result.reset_index(drop=True, inplace=True)
    
    return df_result

In [ ]:
def word_count_result(df,column_list): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    
    return  df_word_count

In [ ]:
def word_in_wordgroup_simple(df, source_column, target_column, word_sample_num):

    '''word_in_wordgroup(df, "word", "twogram"):
       df is dataframe, source_column and target_column are 
       dataframe column string name. source_column convert list
       values that are in target column.
    '''
    
    df_select = df[[f"{target_column}"]].dropna()
    df_result = pd.DataFrame()
    for i in df[f"{source_column}"].dropna():
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(word_sample_num)    
        except:
            pass        
        word_in_word_cluster.insert(0,f"{source_column}",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

In [ ]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

In [ ]:
df_word_select = df_word_all.iloc[word_start:word_end,]

In [ ]:
word_list = df_word_select["word"].to_list()

In [ ]:
df_two_three = pd.read_excel("/home/kurubal/Downloads/Twogram Threegram.xlsx")
df_two_three

In [ ]:
word_count_result(df_two_three,["threegram"])

In [ ]:
df_two_three["twogram"].count()

In [ ]:
df_two_three["threegram"].count()

In [ ]:
df_two_in_three = word_in_wordgroup_simple(df_two_three, "twogram","threegram",threegram_sample)
df_two_in_three

In [ ]:
set_all_twogram = set(df_two_three["twogram"].dropna().to_list())
set_twogram_in_threegram = set(df_two_in_three["twogram"].to_list())

In [ ]:
set_all_twogram.difference(set_twogram_in_threegram)

In [ ]:
set_threegram_cover_twogram = set(df_two_in_three["threegram"].to_list())
df_selected_threegram = pd.DataFrame(set_threegram_cover_twogram,columns=["threegram"])
df_selected_threegram

In [ ]:
df_selected_threegram.to_excel("Threegram_Selected.xlsx", index=False)

In [ ]:
word_count_result(df_selected_threegram,["threegram"])

In [ ]:
df_threegram_word_usage_select = word_usage_result(word_list, df_selected_threegram, "threegram", word_use_num_min, word_use_num_max)
df_threegram_word_usage_select

In [ ]:
word_count_result(df_threegram_word_usage_select,["threegram"])